position formula:
`mujoco_pos_child = 2 * unity_pos_child - size_parent + size_child - 2 * unity_pos_immediate_parent`

Where:
- `pos_child` is the vertical position of the child
- `size_parent` is the vertical size of the immediate parent segment
- `size_child` is the vertical size of the child segment
- `pos_immediate_parent` is the vertical position of the parent

We are doing this because in Unity, gameobjects are centered on the center of the bottom face, but in Mujoco, the bodies are centered in the very center of the object

For other axes, 
`2 * (pos_child  - pos_immediate_parent)`

In [2]:
import json

entries = {}

filename = 'blueprint2.json'
with open(filename) as f:
    data = json.load(f)
    for entry in data:
        id = entry["UniqueId"]
        entries[id] = entry

    sz_root_y = entries[0]["Size"]["y"]
    print(f"sz_root_y: {sz_root_y}")
    for entry in data:
        parentId = entry["ParentUniqueId"]
        if parentId is None:
            # I forgot what to do here
            continue

        sz_parent_y = entries[parentId]["Size"]["y"]

        # Calculate X coordinate
        uty_pos_child_x = entry["Position"]["x"]
        uty_pos_immediate_parent = entries[parentId]["Position"]["x"]
        muj_pos_child_x = 2 * (uty_pos_child_x - uty_pos_immediate_parent)
        print(f"Entry: {entry['UniqueId']}:\n Pos X\t"\
            "muj_pos_child_x = 2 * (uty_pos_child_x - uty_pos_immediate_parent)\n\t"\
            f"= 2 * ({uty_pos_child_x} - {uty_pos_immediate_parent})\n\t"\
            f"= {muj_pos_child_x}")
        
        # Calculate Z coordinate
        uty_pos_child_z = entry["Position"]["z"]
        uty_pos_immediate_parent = entries[parentId]["Position"]["z"]
        muj_pos_child_z = 2 * (uty_pos_child_z - uty_pos_immediate_parent)
        print(f" Pos Z\t"\
            "muj_pos_child_z = 2 * (uty_pos_child_z - uty_pos_immediate_parent)\n\t"\
            f"= 2 * ({uty_pos_child_z} - {uty_pos_immediate_parent})\n\t"\
            f"= {muj_pos_child_z}")
        
        # Calculate Y coordinate
        sz_child_y = entry["Size"]["y"]
        uty_pos_child_y = entry["Position"]["y"]
        uty_pos_immediate_parent = entries[parentId]["Position"]["y"]
        # muj_pos_child_y = 2 * uty_pos_child_y - sz_root_y + sz_child_y - uty_pos_immediate_parent
        # muj_pos_child_y = 2 * uty_pos_child_y - sz_root_y + sz_child_y - 2 * uty_pos_immediate_parent
        muj_pos_child_y = 2 * uty_pos_child_y - sz_parent_y + sz_child_y - 2 * uty_pos_immediate_parent

        # Print results
        print(f" Pos Y\t"\
            "muj_pos_child_y = 2 * uty_pos_child_y - sz_root_y + sz_child_y - 2 * uty_pos_immediate_parent\n\t"\
            f"= 2 * {uty_pos_child_y} - {sz_root_y} + {sz_child_y} - {uty_pos_immediate_parent}\n\t"\
            f"= {muj_pos_child_y}")


sz_root_y: 1.3171674
Entry: 1:
 Pos X	muj_pos_child_x = 2 * (uty_pos_child_x - uty_pos_immediate_parent)
	= 2 * (-0.373039246 - 0.0)
	= -0.746078492
 Pos Z	muj_pos_child_z = 2 * (uty_pos_child_z - uty_pos_immediate_parent)
	= 2 * (0.06939888 - 0.0)
	= 0.13879776
 Pos Y	muj_pos_child_y = 2 * uty_pos_child_y - sz_root_y + sz_child_y - 2 * uty_pos_immediate_parent
	= 2 * 1.3171674 - 1.3171674 + 0.419512331 - 0.0
	= 1.736679731
Entry: 2:
 Pos X	muj_pos_child_x = 2 * (uty_pos_child_x - uty_pos_immediate_parent)
	= 2 * (0.390892744 - 0.0)
	= 0.781785488
 Pos Z	muj_pos_child_z = 2 * (uty_pos_child_z - uty_pos_immediate_parent)
	= 2 * (-0.06310558 - 0.0)
	= -0.12621116
 Pos Y	muj_pos_child_y = 2 * uty_pos_child_y - sz_root_y + sz_child_y - 2 * uty_pos_immediate_parent
	= 2 * 0.153632462 - 1.3171674 + 0.5905807 - 0.0
	= -0.419321776
Entry: 3:
 Pos X	muj_pos_child_x = 2 * (uty_pos_child_x - uty_pos_immediate_parent)
	= 2 * (0.350706816 - 0.390892744)
	= -0.08037185600000007
 Pos Z	muj_pos_child_

- have to nest segments in xml
- figure out the rule to use for calculating joint positions (are we currently just multiplying each coordinate by -1?)
- the parts we modify through our script:
    <body name="segMODIFY" pos="MODIFY MODIFY MODIFY">
      <joint name="segMODIFY_to_MODIFY" range="-75 75" type="hinge" axis="MODIFY MODIFY MODIFY" pos="MODIFY MODIFY MODIFY"/>
      <geom name="segMODIFY_geom" type="box" pos="0 0 0" size="MODIFY MODIFY MODIFY" euler="MODIFY MODIFY MODIFY" rgba="1 1 1 0"/>
      <site name="segMODIFY_site" type="box" pos="0.1 0.1 0" size="0.09 0.09 0.24" zaxis="0.2 0.2 0" rgba="1 1 0 0"/>
    </body>    

steps for creating a segment (manually rn):
- in ant.xml,
  - paste in template from above in the right XML section
  - change the MODIFYs that have to do with the segment and parent ids
  - add the motor to the actuator section
  - add the sensor to the sensor section
  - add all newly created contact exclusion pairs
  - fill in all the MODIFYs
- in ant.py,
  - in marker_geoms(self), add the new segment's geom